<a href="https://colab.research.google.com/github/Farmhouse121/Financial-Data-Science-in-Python/blob/main/The%20Market's%20Not%20Normal/The_Market's_Not_Normal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Market's Not Normal
---
## Preamble

Hi, this is the first "walk-through" in my project _Financial Data Science in Python_. You can read about what this project is in my article on Medium [Financial Data Science in Python](https://medium.com/adventures-in-data-science/financial-data-science-in-python-ee66dab460cf).

To execute the code in this notebook, you're going to need to upload the my_library.py file from the associated GitHub repository (because Google will delete it when the virtual machine goes away, and so it's likely no longer available here). This is easy, but nothing will work without it.

Once that is done, execute the next cell...

In [1]:
from my_library import *

22:06:30 Starting...
22:06:30 Installing yfinance into Google notebook...
22:06:36 Installing arch into Google notebook...
22:06:44 Initialized.
